# Imports

In [1]:
# import the required libraries
import os
import numpy as np
import base64
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from PIL import Image
from io import BytesIO

2024-02-27 19:52:58.897418: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-27 19:52:59.861672: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Import pre-trained model

In [2]:
pre_trained_model = tf.keras.applications.MobileNetV2()
pre_trained_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [3]:
len(pre_trained_model.layers)

156

# Get Labels

In [4]:
LABELS_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt', LABELS_URL)
class_names = np.array(
    open(labels_path).read().splitlines()
)[1:]

print('Labels shape:', class_names.shape)

print(class_names)

Labels shape: (1000,)
['tench' 'goldfish' 'great white shark' 'tiger shark' 'hammerhead'
 'electric ray' 'stingray' 'cock' 'hen' 'ostrich' 'brambling' 'goldfinch'
 'house finch' 'junco' 'indigo bunting' 'robin' 'bulbul' 'jay' 'magpie'
 'chickadee' 'water ouzel' 'kite' 'bald eagle' 'vulture' 'great grey owl'
 'European fire salamander' 'common newt' 'eft' 'spotted salamander'
 'axolotl' 'bullfrog' 'tree frog' 'tailed frog' 'loggerhead'
 'leatherback turtle' 'mud turtle' 'terrapin' 'box turtle' 'banded gecko'
 'common iguana' 'American chameleon' 'whiptail' 'agama' 'frilled lizard'
 'alligator lizard' 'Gila monster' 'green lizard' 'African chameleon'
 'Komodo dragon' 'African crocodile' 'American alligator' 'triceratops'
 'thunder snake' 'ringneck snake' 'hognose snake' 'green snake'
 'king snake' 'garter snake' 'water snake' 'vine snake' 'night snake'
 'boa constrictor' 'rock python' 'Indian cobra' 'green mamba' 'sea snake'
 'horned viper' 'diamondback' 'sidewinder' 'trilobite' 'harvest

# test single image

In [5]:
testfile = "samples/banana.jpeg"

# Open image and format to RGB and size
img = Image.open(testfile).convert('RGB')
img = img.resize((224, 224), Image.LANCZOS)
img = np.asarray(img)

# Convert to Numpy
mydata = np.empty((1, 224, 224, 3))
mydata[0] = img

# Convert from 0..255 values to -1..1 values
ppdata = preprocess_input(mydata)

# Run prediction
myprediction = pre_trained_model.predict(ppdata)

print(myprediction)

1/1 [==============================] - 1s 725ms/step
[[3.12336109e-04 8.43091620e-05 2.33669183e-04 6.83255348e-05
  4.59889852e-05 9.37240693e-05 1.08052016e-04 1.20842982e-04
  1.13170281e-04 1.90412291e-04 7.11781977e-05 2.12286192e-04
  5.02765106e-05 5.03354240e-05 7.65430814e-05 5.14213425e-05
  2.35828440e-04 1.46128994e-04 2.16753484e-04 2.61551002e-04
  3.40776933e-05 5.55607199e-04 2.30267513e-04 4.84502903e-04
  1.05399922e-04 1.64804122e-04 5.22529299e-04 1.32508096e-04
  1.49350017e-04 1.68384751e-04 8.03904622e-05 2.35792671e-04
  3.21071857e-04 6.41405568e-05 1.32504429e-04 6.33691161e-05
  1.27495732e-04 2.72081408e-04 1.06257277e-04 1.00206154e-04
  1.28727144e-04 2.88347219e-04 1.49329368e-04 3.27090005e-04
  2.01140385e-04 2.73668324e-04 1.75947120e-04 9.38120647e-05
  1.71596490e-04 7.38992894e-05 7.95123269e-05 1.00122757e-04
  2.20939197e-04 1.60699288e-04 3.07491136e-04 2.46624579e-04
  4.31234745e-04 1.21524616e-04 2.07888643e-05 4.01248340e-04
  2.26487711e-04 

# Highest prediction

In [6]:
print(np.argmax(myprediction, axis=1))

[954]


# Prediction label

In [7]:
print(class_names[np.argmax(myprediction, axis=1)])

['banana']


# Save Base model

In [8]:
# We save the model in order to genereate a default signature we can later modify
tf.saved_model.save(pre_trained_model, "./models/base/1")


INFO:tensorflow:Assets written to: ./models/base/1/assets


INFO:tensorflow:Assets written to: ./models/base/1/assets


# Save model with Base64 Signature

### Load labels

In [9]:
# print(class_names)
labels = tf.constant([class_names])
# print(labels)

### Load base model, add signature and save

In [10]:
smodel = tf.saved_model.load("./models/base/1")

# This is the current signature, that only accepts image tensors as input
signature = smodel.signatures["serving_default"]
print(signature)

# obtain the key name of the output (typically 'dense_X')
keyOutput = next(iter(signature.structured_outputs))

@tf.function()
def my_predict(image_b64):

    # get content
    content = image_b64[0]
    
    # decode image    
    image = tf.image.decode_jpeg(content,channels=3)
    # tf.compat.v1.enable_eager_execution()
    
    # resize image
    image = tf.image.resize(image, [224, 224])
    
    # expand dimension to match signature
    image = tf.expand_dims(image, 0)
    
    # set values in -1..1 range
    image = preprocess_input(image)
        
    # execute prediction
    prediction = signature(image)

    # obtain index of maximum probability prediction
    idx = tf.argmax(prediction[keyOutput],axis=1)
    
    # obtain the label for given index
    label = tf.gather(labels, idx, batch_dims=1)

    # obtain probability from Tensor
    probability = prediction[keyOutput][0,idx[0]]
    
    # combine result in String Tensor format with [label,probability]
    result = tf.concat([label, [tf.as_string(probability)]], axis=0)
    
    # return result_tensor
    return result

# Create new signature, to read b64 images
new_signature = my_predict.get_concrete_function(
    image_b64=tf.TensorSpec(name="image_b64", shape=[1], dtype=tf.string)
)

# Save model with Base64 input signature
tf.saved_model.save(
    smodel,
    export_dir="./models/redbag/1",
    signatures=new_signature
)

ConcreteFunction signature_wrapper(*, input_1)
  Args:
    input_1: float32 Tensor, shape=(None, 224, 224, 3)
  Returns:
    {'predictions': <1>}
      <1>: float32 Tensor, shape=(None, 1000)
INFO:tensorflow:Assets written to: ./models/redbag/1/assets


INFO:tensorflow:Assets written to: ./models/redbag/1/assets


# Test Base64 Model with single image

In [11]:
smodel = tf.saved_model.load("./models/redbag/1")

# Load model's signature
signature = smodel.signatures["serving_default"]

print(signature)

ConcreteFunction signature_wrapper(*, image_b64)
  Args:
    image_b64: string Tensor, shape=(1,)
  Returns:
    {'output_0': <1>}
      <1>: string Tensor, shape=(2,)


### Run Inference

In [12]:
testfile = "./samples/banana.jpeg"

# load test image
content = tf.io.read_file(testfile)

# reshape to signature's expected dimensions
content = tf.reshape(content, shape = [1])

# print(tf.print(content, summarize=3))

# obtain signature
f = smodel.signatures["serving_default"]

# run prediction
myprediction = f(image_b64=content)
print(myprediction)

{'output_0': <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'banana', b'0.808100'], dtype=object)>}
